# Day 4
## Part 1

> Count the number of valid passports - those that have all required fields. Treat cid as optional. In your batch file, how many passports are valid?

In [2]:
from aocd.models import Puzzle
import numpy as np
import re

In [3]:
puzzle = Puzzle(year=2020, day=4)

I'm creating a class for Passports first up:

In [4]:
class Passport:
    def __init__(self, attr_dict):
        self.birth_year = attr_dict.get('byr', None)
        self.issue_year = attr_dict.get('iyr', None)
        self.exp_year = attr_dict.get('eyr', None)
        self.height = attr_dict.get('hgt', None)
        self.hair_color = attr_dict.get('hcl', None)
        self.eye_color = attr_dict.get('ecl', None)
        self.passport_id = attr_dict.get('pid', None)
        self.country_id = attr_dict.get('cid', None)
    
    def is_valid(self):
        if self.birth_year and self.issue_year and self.exp_year and self.height and self.hair_color and self.eye_color and self.passport_id:
            return True
        return False

Instead of a `Builder` class i'll use this function:

In [5]:
def make_passport(s):
    prop_dict = {}
    for line in s.splitlines():
        for prop in line.split(' '):
            k, v = prop.split(':')
            prop_dict[k] = v
    
    return Passport(prop_dict)


## Test
s = """ecl:gry pid:860033327 eyr:2020 hcl:#fffffd
byr:1937 iyr:2017 cid:147 hgt:183cm"""

assert make_passport(s).is_valid()

s = """iyr:2013 ecl:amb cid:350 eyr:2023 pid:028048884
hcl:#cfa07d byr:1929"""

assert not make_passport(s).is_valid()

s = """hcl:#ae17e1 iyr:2013
eyr:2024
ecl:brn pid:760753108 byr:1931
hgt:179cm"""

assert make_passport(s).is_valid()


and this driver code to get the valid passports:

In [6]:
passport_string = ""
valid_passports = 0
for line in puzzle.input_data.splitlines():
    if line == "":
        if make_passport(passport_string).is_valid():
            valid_passports += 1
        passport_string = ""
    else:
        passport_string += line
        passport_string += '\n'

if make_passport(passport_string).is_valid():
    valid_passports += 1

valid_passports

228

In [7]:
puzzle.answer_a = valid_passports

## Part 2

I decided to extend the previous class and convert the fields into objects instead of strings; each with their own validation implementations. I also used a dictionary of lambda functions to implement a `Factory` for my fields

In [10]:
class Field:
    def __init__(self, s):
        self.field = s

    def is_valid(self):
        pass

class Year(Field):
    def is_valid(self):
        if re.match(r'^[0-9]{4}$', self.field):
            return True
        return False

class BirthYear(Year):
    def is_valid(self):
        return super().is_valid() and int(self.field) >= 1920 and int(self.field) <= 2002
            
class IssueYear(Year):
    def is_valid(self):
        return super().is_valid() and int(self.field) >= 2010 and int(self.field) <= 2020
    
class ExpYear(Year):
    def is_valid(self):
        return super().is_valid() and int(self.field) >= 2020 and int(self.field) <= 2030

class Height(Field):
    def is_valid(self):
        m = re.match(r'(^[0-9]*)(cm|in)$', self.field)
        if not m:
            return False
        if m.groups()[1] == 'cm':
            return int(m.groups()[0]) >= 150 and int(m.groups()[0]) <= 193
        if m.groups()[1] == 'in':
            return int(m.groups()[0]) >= 59 and int(m.groups()[0]) <= 76

class HairColor(Field):
    def is_valid(self):
        if re.match(r'^#[a-fA-F0-9]{6}$', self.field):
            return True
        return False

class EyeColor(Field):
    def is_valid(self):
        if re.match(r'^(amb|blu|brn|gry|grn|hzl|oth)$', self.field):
            return True
        return False

class PassportId(Field):
    def is_valid(self):
        if re.match(r'^[0-9]{9}$', self.field):
            return True
        return False

class StrictPassport(Passport):
    def is_valid(self):
        if not super().is_valid():
            return False
        
        fields = [
            self.birth_year,
            self.issue_year,
            self.exp_year,
            self.height,
            self.hair_color,
            self.eye_color,
            self.passport_id
        ]

        for field in fields:
            if not field.is_valid():
                return False
        
        return True

field_factory = {
    'byr': lambda x: BirthYear(x),
    'iyr': lambda x: IssueYear(x),
    'eyr': lambda x: ExpYear(x),
    'hgt': lambda x: Height(x),
    'hcl': lambda x: HairColor(x),
    'ecl': lambda x: EyeColor(x),
    'pid': lambda x: PassportId(x),
    'cid': lambda x: x
}

def make_strict_passport(s):
    prop_dict = {}
    for line in s.splitlines():
        for prop in line.split(' '):
            k, v = prop.split(':')
            prop_dict[k] = field_factory[k](v)
    
    return StrictPassport(prop_dict)

In [11]:
passport_string = ""
valid_passports = 0
for line in puzzle.input_data.splitlines():
    if line == "":
        if make_strict_passport(passport_string).is_valid():
            valid_passports += 1
        passport_string = ""
    else:
        passport_string += line
        passport_string += '\n'

if make_strict_passport(passport_string).is_valid():
    valid_passports += 1

valid_passports

175

In [12]:
puzzle.answer_b = valid_passports

That's the right answer!  You are one gold star closer to saving your vacation.You have completed Day 4! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].
